In [15]:
import pandas as pd
import requests
import json
from pandasql import sqldf

In [35]:
class KChallenge():  #Classe KChallenge, onde instanciarei como dataframe principal o arquivo xlsx, que sofrerá as maiores transformações
    def __init__(self, df):  #método construtor, recebendo o dataframe como atribuito
        self.df = df
    
    def getdataframe(self):  #retorna o dataframe
        return self.df
    
    def showdataframe(self):  #mostra o dataframe
        print(self.df)
    
    def nocommas(self, column):  #substitui caracteres ',' por '.'
        self.df[column] = self.df[column].str.replace(',', '.')

    def pivoting(self, index, column, value):  #pivota o dataframe para transformar o conteúdo de linhas em colunas
        self.df = pd.pivot_table(data = self.df, index = index, columns=column, values=value, aggfunc='first')
        self.df = self.df.reset_index()
        
    def nonulls(self, column_n_values: dict):  #trata nulos, recebe um dicionário com Coluna:Valor a ser preenchido
        self.df = self.df.fillna(column_n_values)  #e preenche com esses valores os nulos destas colunas

    def remap(self, df_alias, remap: dict, column):  #método para remapear colunas e substituir uma coluna por outros valores
        self.df[column].map(remap)
    
    def total_produtos(self, keycolumn):  #retorna o numero total de produtos únicos
        return self.df[keycolumn].nunique()

    def categorias_produtos_unicos(self, groupcolumn, targetcolumn):  #retorna o total de produtos únicos por categoria
        prod_uniq = self.df.groupby(groupcolumn)[targetcolumn].nunique()
        for r in dict(prod_uniq):
            print(f'Categoria {r} contém {prod_uniq[r]} produtos únicos')

In [36]:
def getautos(post_url, get_url, header):  #Função criada para conseguir o dataframe através de uma conexão via API
    r = requests.post(url = post_url, json=header)  #metodo post (requisição http), com os parametros necessários
    if r.status_code == 201:  #caso a resposta seja positiva
        r1 = r.json()  #recebe o conteudo da requisição
        token = r1['API Token']  #variável token recebe o valor do dicionário na chave "API Token"
        r = requests.get(url = get_url+token)  #faz uma nova requisição http (get) para, com o Token gerado para meu usuário, obter o próximo dataframe
        if r.status_code == 200:  #caso a resposta seja positiva
            conteudo = r.json()  #recebo o conteúdo da resposta na variável conteudo
            lista = []  #lista para criar a próxima tabela
            for json in conteudo:  #percorrendo o conteúdo
                df = pd.DataFrame(json, index=[0])  #transformando o json em um dataframe
                lista.append(df)  #adicionando o dataframe ao fim da lista
            df = pd.concat(lista, ignore_index=True)  #transforma todo o conteúdo lido dos jsons em uma tabela no formato dataframe
            return df  #retorna o dataframe
        
def transform_autos(df, column, split, nulldrop):  #como temos uma coluna com diversos conteúdos separados por caractere
    df[column] = df[column].str.split(split)  #criaremos novas tuplas a partir destes conteúdos
    df = df.explode(column)  #explodindo a coluna para criar as novas tuplas
    df = df.dropna(subset=[nulldrop])  #dropa colunas que nao tiverem valor para uma coluna escolhida
    return df  #retorna o dataframe

##### Optei por não criar uma classe main para deixar tudo no notebook para melhor visualização

##### Abaixo, criarei um objeto e realizarei o solicitado para o desafio

In [41]:
df = pd.read_excel('karhub_autoparts_1.xlsx')
kc = KChallenge(df)
kc.nocommas('Valor Atributo')
index=['Nome SKU', 'Fabricante', 'Código', 'Composição', 'Categoria']
column='Nome Atributo'
value='Valor Atributo'
replace_nulls = {'Motor': '-', 'Número de Espirais': 0}
kc.pivoting(index, column, value)
kc.nonulls(replace_nulls)
df = kc.getdataframe()
df

Nome Atributo,Nome SKU,Fabricante,Código,Composição,Categoria,Altura (cm),Comprimento (cm),Largura (cm),Motor,Número de Espirais,Peso Bruto (Kg),Peso Líquido (Kg),Posição
0,Catalisador,KarHub,CH0613,UNITARY,Sonda Lambda,25,25,40,-,6.43,4.5,2.0,Traseira
1,Catalisador,KarHub,FO-0024,UNITARY,Sonda Lambda,25,25,40,-,10.6,4.5,2.0,Traseira
2,Catalisador,KarHub,FT0009,UNITARY,Sonda Lambda,25,25,40,-,7.36,4.5,2.0,Traseira
3,Catalisador,KarHub,FT0367M,UNITARY,Sonda Lambda,25,25,40,-,6,4.5,2.0,Traseira
4,Catalisador,KarHub,I-FO0307-M,UNITARY,Sonda Lambda,25,25,40,-,9.85,4.5,2.0,Traseira
...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,Tambor de freio Para GNV,KarHub,J-FT1020-FX,UNITARY,Freio,25,25,40,-,3,4.5,2.0,Traseira
476,Trizeta,KarHub,07402767.0,UNITARY,Transmissão,25,25,40,-,10.75,4.5,2.0,Traseira
477,Trizeta,KarHub,CH-0017,UNITARY,Transmissão,25,25,40,-,7.3,4.5,2.0,Traseira
478,Trizeta,KarHub,CH0347,UNITARY,Transmissão,25,25,40,-,7.2,4.5,2.0,Traseira


In [5]:
post_url = 'https://api-data-engineer-test-3bqvkbbykq-uc.a.run.app/user/'
header = {
  "full_name": "Dino da Silva Sauro",
  "email": "spinodino@dinoland.com"
}
get_url = 'https://api-data-engineer-test-3bqvkbbykq-uc.a.run.app/token='

df_api = getautos(post_url=post_url, get_url=get_url, header=header)


In [38]:
df_api = transform_autos(df_api,'Complemento','|','Marca')
df_api

,Fabricante,Código Fabricante,Marca,Modelo,Ano,Complemento,MarcaModelo
0,KarHub,FO0352,FORD,FIESTA,1996.0,1.0 MPI 8V CLASS 2P,FORDFIESTA
0,KarHub,FO0352,FORD,FIESTA,1996.0,1.0 MPI 8V - 4P,FORDFIESTA
0,KarHub,FO0352,FORD,FIESTA,1996.0,1.0 MPI 8V - 2P,FORDFIESTA
0,KarHub,FO0352,FORD,FIESTA,1996.0,1.3 MPI 8V CLX 4P,FORDFIESTA
0,KarHub,FO0352,FORD,FIESTA,1996.0,1.0 MPI 8V CLASS 4P,FORDFIESTA
...,...,...,...,...,...,...,...
4212,KarHub,I-VW0468,AUDI,A3,2006.0,1.8 20V - 4P,AUDIA3
4212,KarHub,I-VW0468,AUDI,A3,2006.0,1.6 8V - 4P,AUDIA3
4212,KarHub,I-VW0468,AUDI,A3,2006.0,1.8 TURBO 20V 150CV - 4P,AUDIA3
4212,KarHub,I-VW0468,AUDI,A3,2006.0,1.8 20V - 2P,AUDIA3


In [39]:
df_alias = pd.read_csv('Karhub-alias.csv')
df_alias

,Fabricante,Código Fabricante,Alias
0,KarHub,FO0352,fo0352
1,KarHub,FO0353,fo0353
2,KarHub,FT0417,ft0417
3,KarHub,VW0127,vw0127
4,KarHub,PE0452,pe0452
...,...,...,...
1245,KarHub,I-VW0468,VW0468
1246,KarHub,I-CH0512,I-CH0512
1247,KarHub,I-CH0512,ich0512
1248,KarHub,I-MI0611,I-MI0611


In [40]:
remap = dict(zip(df_alias['Alias'], df_alias['Código Fabricante']))
kc.remap(df_alias, remap,'Código')
kc.showdataframe()

Nome Atributo                  Nome SKU Fabricante       Código Composição  \
0                           Catalisador     KarHub       CH0613    UNITARY   
1                           Catalisador     KarHub      FO-0024    UNITARY   
2                           Catalisador     KarHub       FT0009    UNITARY   
3                           Catalisador     KarHub      FT0367M    UNITARY   
4                           Catalisador     KarHub   I-FO0307-M    UNITARY   
..                                  ...        ...          ...        ...   
475            Tambor de freio Para GNV     KarHub  J-FT1020-FX    UNITARY   
476                             Trizeta     KarHub   07402767.0    UNITARY   
477                             Trizeta     KarHub      CH-0017    UNITARY   
478                             Trizeta     KarHub       CH0347    UNITARY   
479                             Trizeta     KarHub     I-TR0517    UNITARY   

Nome Atributo     Categoria Altura (cm) Comprimento (cm) Largur

In [31]:
print(f'Existem {kc.total_produtos("Código")} produtos únicos')

Existem 480 produtos únicos


In [32]:
df_api['MarcaModelo'] = df_api['Marca'] + df_api['Modelo']
# dfq2 = dfq2.drop(['Código Fabricante','Marca', 'Modelo'], axis=1)
# dfq2.drop_duplicates()
df_api

,Fabricante,Código Fabricante,Marca,Modelo,Ano,Complemento,MarcaModelo
0,KarHub,FO0352,FORD,FIESTA,1996.0,1.0 MPI 8V CLASS 2P,FORDFIESTA
0,KarHub,FO0352,FORD,FIESTA,1996.0,1.0 MPI 8V - 4P,FORDFIESTA
0,KarHub,FO0352,FORD,FIESTA,1996.0,1.0 MPI 8V - 2P,FORDFIESTA
0,KarHub,FO0352,FORD,FIESTA,1996.0,1.3 MPI 8V CLX 4P,FORDFIESTA
0,KarHub,FO0352,FORD,FIESTA,1996.0,1.0 MPI 8V CLASS 4P,FORDFIESTA
...,...,...,...,...,...,...,...
4212,KarHub,I-VW0468,AUDI,A3,2006.0,1.8 20V - 4P,AUDIA3
4212,KarHub,I-VW0468,AUDI,A3,2006.0,1.6 8V - 4P,AUDIA3
4212,KarHub,I-VW0468,AUDI,A3,2006.0,1.8 TURBO 20V 150CV - 4P,AUDIA3
4212,KarHub,I-VW0468,AUDI,A3,2006.0,1.8 20V - 2P,AUDIA3


In [33]:
veic_uniq = df_api.groupby('Código Fabricante')['MarcaModelo'].nunique()
for r in dict(veic_uniq):
    print(f'Produto {r} contém {veic_uniq[r]} veículos únicos')

Produto FO0044 contém 1 veículos únicos
Produto FO0054 contém 1 veículos únicos
Produto FO0191 contém 2 veículos únicos
Produto FO0208 contém 1 veículos únicos
Produto FO0231 contém 3 veículos únicos
Produto FO0300 contém 3 veículos únicos
Produto FO0301 contém 3 veículos únicos
Produto FO0306 contém 4 veículos únicos
Produto FO0352 contém 1 veículos únicos
Produto FO0353 contém 1 veículos únicos
Produto FT0007 contém 2 veículos únicos
Produto FT0291 contém 1 veículos únicos
Produto FT0366 contém 1 veículos únicos
Produto FT0371 contém 1 veículos únicos
Produto FT0372 contém 1 veículos únicos
Produto FT0378 contém 1 veículos únicos
Produto FT0380 contém 1 veículos únicos
Produto FT0417 contém 1 veículos únicos
Produto FT0422 contém 1 veículos únicos
Produto FT0423 contém 1 veículos únicos
Produto I-CH0005 contém 1 veículos únicos
Produto I-CH0018 contém 1 veículos únicos
Produto I-CH0019 contém 4 veículos únicos
Produto I-CH0020 contém 1 veículos únicos
Produto I-CH0028 contém 1 veícul

In [34]:
kc.categorias_produtos_unicos('Categoria', 'Código')

Categoria Freio contém 18 produtos únicos
Categoria Mola Helicoidal contém 417 produtos únicos
Categoria Sonda Lambda contém 22 produtos únicos
Categoria Transmissão contém 23 produtos únicos
